In [1]:
import cv2
import numpy as np
from PIL import Image
import copy

In [2]:
virat_video_path = '/Users/wasifraza/projects/mask-extraction/virat.mp4'
virat_cap = cv2.VideoCapture(virat_video_path)
virat_fps = int(virat_cap.get(cv2.CAP_PROP_FPS))
virat_frame_width = int(virat_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
virat_frame_height = int(virat_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

personal_video_path = '/Users/wasifraza/projects/mask-extraction/IMG_5375.mp4'
personal_cap = cv2.VideoCapture(personal_video_path)
personal_fps = int(personal_cap.get(cv2.CAP_PROP_FPS))
personal_frame_width = int(personal_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
personal_frame_height = int(personal_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_path = 'merged_video_updated.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, personal_fps, (personal_frame_width, personal_frame_height))

output_mask_path = 'merged_video_mask.mp4'
out_mask = cv2.VideoWriter(output_mask_path, fourcc, personal_fps, (personal_frame_width, personal_frame_height))

# print((virat_frame_width, virat_frame_height))
output_croppedmask_path = 'fg_mask.mp4'
output_croppedmask = cv2.VideoWriter(output_croppedmask_path, fourcc, personal_fps, (virat_frame_width, virat_frame_height))

output_cropped_virat_path = 'cropped_virat.mp4'
out_cropped_virat = cv2.VideoWriter(output_cropped_virat_path, fourcc, personal_fps, (686, 286))

# for _ in range(100):
while True:
    # Read frames from personal video
    ret_personal, personal_frame = personal_cap.read()
    if not ret_personal:
        break
    
    # Read frames from Virat Kohli video
    ret_virat, virat_frame = virat_cap.read()
    if not ret_virat:
        virat_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        _, virat_frame = virat_cap.read()  # Re-read the first frame if video ends

    mask = cv2.inRange(virat_frame, (0, 150, 0), (100, 255, 100))  # Define range of green color
    mask_inv = cv2.bitwise_not(mask)
    save_mask = cv2.cvtColor(mask_inv, cv2.COLOR_GRAY2BGR)
    # cv2.imwrite('test.png', save_mask)
    # print(save_mask.shape)
    # break

    output_croppedmask.write(save_mask)
    contours, _ = cv2.findContours(mask_inv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contours[0])
    idx=1
    while (w<5 or h<5) and len(contours)>idx:
        x, y, w, h = cv2.boundingRect(contours[idx])
        idx+=1
        # print(x,y,w,h)
    if (w==1 or h==1):
        print(w,h)
        continue
    # print(w,h)
    cropped_virat = virat_frame[y:y+h, x:x+w]
    cropped_mask = mask[y:y+h, x:x+w]
    aspect_ratio = cropped_virat.shape[1] / cropped_virat.shape[0]

    new_height = personal_frame.shape[0]
    x_offset = 400  # X coordinate of the top-left corner of the ROI
    y_offset_upper = 400
    y_offset_bottom = 150
    adjusted_height = new_height - y_offset_upper - y_offset_bottom

    adjusted_width = int(adjusted_height * aspect_ratio)

    # print(adjusted_height, adjusted_width)

    adjusted_mask = cv2.resize(cropped_mask, (adjusted_width, adjusted_height))
    adjusted_virat = cv2.resize(cropped_virat, (adjusted_width, adjusted_height))
    # Image.fromarray(cropped_virat)

    personal_frame_adjusted = copy.copy(personal_frame)
    personal_frame_extraction = personal_frame_adjusted[y_offset_upper:y_offset_upper + adjusted_height,  x_offset: x_offset + adjusted_width]

    # print(y_offset_upper + adjusted_height, x_offset + adjusted_width)
    # print(personal_frame_extraction.shape)
    # print(adjusted_mask.shape)
    adjusted_virat[adjusted_mask != 0] = personal_frame_extraction[adjusted_mask != 0]
    # print(adjusted_virat.shape)


    # Generate mask for chroma keying
    personal_frame_adjusted[y_offset_upper:y_offset_upper + adjusted_height,  x_offset: x_offset + adjusted_width] = adjusted_virat

    complete_mask = np.zeros((personal_frame_height, personal_frame_width), dtype=np.uint8)
    complete_mask[y_offset_upper:y_offset_upper + adjusted_height,  x_offset: x_offset + adjusted_width] = cv2.bitwise_not(adjusted_mask)
    mask_image_3ch = cv2.cvtColor(complete_mask, cv2.COLOR_GRAY2BGR)
    out_mask.write(mask_image_3ch)
    out.write(personal_frame_adjusted)
    out_cropped_virat.write(cropped_virat)

personal_cap.release()
virat_cap.release()
out.release()
out_mask.release()
out_cropped_virat.release()
output_croppedmask.release()
